In [33]:
!pip install --force-reinstall -U -r requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 23.0.0 requires cryptography<40,>=38.0.0, but you have cryptography 46.0.1 which is incompatible.
conda 23.5.2 requires ruamel-yaml<0.18,>=0.11.14, but you have ruamel-yaml 0.18.15 which is incompatible.


In [36]:
import os
import sys

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# Now you can import utils
import utils

In [1]:
%%writefile enhanced_strands_claude_with_nova.py
# Example: Enhanced Strands agent with Nova Act browser tool
from strands import Agent, tool
from strands_tools import calculator
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct
import json

app = BedrockAgentCoreApp()

# Original custom tool 
@tool
def weather():
    """ Get weather """ 
    return "sunny"

# NEW: Nova Act browser tool
@tool
def browse_web(instruction: str, starting_url: str = "https://google.com"):
    """Use Nova Act to browse the web and complete web-based tasks
    
    Args:
        instruction: What you want to accomplish on the web
        starting_url: URL to start from
    
    Returns:
        Result of the web browsing task
    """
    try:
        with browser_session("us-east-1") as browser_client:
            ws_url, headers = browser_client.generate_ws_headers()
            
            with NovaAct(
                cdp_endpoint_url=ws_url,
                cdp_headers=headers,
                nova_act_api_key="your-nova-act-api-key",  # Replace with your key
                starting_page=starting_url,
            ) as nova_act:
                result = nova_act.act(instruction)
                return f"Web task completed: {str(result)}"
    except Exception as e:
        return f"Error browsing web: {str(e)}"

# Initialize enhanced agent with Nova Act
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(model_id=model_id)

agent = Agent(
    model=model,
    tools=[calculator, weather, browse_web],  # Now includes web browsing!
    system_prompt="You're a helpful assistant. You can do math calculations, tell the weather, and browse the web to find information."
)

@app.entrypoint
def enhanced_strands_agent(payload):
    """Enhanced agent that can browse the web"""
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

Overwriting enhanced_strands_claude_with_nova.py


In [10]:
from strands import Agent, tool
from strands_tools import calculator
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct
import json

app = BedrockAgentCoreApp()

# Original custom tool 
@tool
def weather():
    """ Get weather """ 
    return "sunny"

# NEW: Nova Act browser tool
@tool
def browse_web(instruction: str, starting_url: str, login_credentials: str, login_password: str):
    """Use Nova Act to browse the web and complete web-based tasks
    
    Args:
        instruction: What you want to accomplish on the web
        starting_url: URL to start from
    
    Returns:
        Result of the web browsing task
    """
    try:
        with browser_session("us-east-1") as browser_client:
            ws_url, headers = browser_client.generate_ws_headers()
            
            with NovaAct(
                cdp_endpoint_url=ws_url,
                cdp_headers=headers,
                nova_act_api_key='50d5f914-8f10-4a14-be4c-2f51f31af940',  # Replace with your key
                starting_page=starting_url,
            ) as nova_act:
                result = nova_act.act(instruction)
                return f"Web task completed: {str(result)}"
    except Exception as e:
        return f"Error browsing web: {str(e)}"

# Initialize enhanced agent with Nova Act
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(model_id=model_id)

agent = Agent(
    model=model,
    tools=[calculator, weather, browse_web],  # Now includes web browsing!
    system_prompt="You're a helpful assistant. You can do math calculations, tell the weather, and browse the web"
)


In [11]:
# 🚀 Try your own queries! Change the prompt below:
payload = {
    "prompt": "Login to the website specified in starting_url using the credentials: login_credentials: str, and login_password: str",
    "starting_url": "https://www.microsoft.com/en/microsoft-365/onedrive/online-cloud-storage?market=af",
    "login_credentials": "user123",
    "login_password": "hunter2"
}

response = agent("Login to the website using the credentials provided and download the Hsinwei_Lin_Resume.",
    starting_url="https://www.microsoft.com/en/microsoft-365/onedrive/online-cloud-storage?market=af",
    login_credentials="user123",
    login_password="hunter2")

I'd be happy to help you login to a website and download the Hsinwei_Lin_Resume, but I need some additional information to proceed with this web browsing task.

For the `browse_web` function, I need:
1. The specific website URL where the login is required (starting_url)
2. The login credentials (username/email)
3. The login password

Could you please provide this information so I can assist with logging in and downloading the resume?

In [16]:
REGION = "us-east-1"

In [25]:
%%writefile lambda_credentials.py
import json
import boto3
from botocore.exceptions import ClientError

def lambda_handler(event, context):
    """Lambda function to retrieve all credentials for a client website
    Args:
        event: Contains client_name for which to retrieve credentials
    Returns:
        Dictionary with all credentials for the requested site
    """
    client_name = event.get('client_name')
    if not client_name:
        return {
            'statusCode': 400,
            'body': json.dumps({
                'success': False,
                'error': 'Missing required parameter: client_name',
                'message': 'Please provide a client_name in the event'
            })
        }

    ssm = boto3.client('ssm')

    def get_secret(parameter_name):
        try:
            response = ssm.get_parameter(Name=parameter_name, WithDecryption=True)
            return response['Parameter']['Value']
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == 'ParameterNotFound':
                raise Exception(f"Parameter '{parameter_name}' not found")
            elif error_code == 'AccessDenied':
                raise Exception(f"Access denied to parameter '{parameter_name}' - check IAM permissions")
            else:
                raise Exception(f"Error retrieving '{parameter_name}': {str(e)}")
        except Exception as e:
            raise Exception(f"Unexpected error retrieving '{parameter_name}': {str(e)}")

    # Construct parameter names dynamically
    login_param = f"{client_name}_Login"
    password_param = f"{client_name}_Password"
    weburl_param = f"{client_name}_WebURL"

    parameter_names = [login_param, password_param, weburl_param]

    try:
        login_credentials = get_secret(login_param)
        password_credentials = get_secret(password_param)
        web_url = get_secret(weburl_param)

        if not all([login_credentials, password_credentials, web_url]):
            missing = []
            if not login_credentials: missing.append(login_param)
            if not password_credentials: missing.append(password_param)
            if not web_url: missing.append(weburl_param)
            raise Exception(f"Missing required parameters: {', '.join(missing)}")

        credentials_response = {
            'starting_url': web_url,
            'login_credentials': login_credentials,
            'login_password': password_credentials,
            'file_info': {
                'upload_path': '/uploads',
                'allowed_extensions': ['.pdf', '.docx', '.txt'],
                'max_file_size': '10MB'
            },
            'retrieved_from': 'AWS_SSM_Parameter_Store',
            'parameters_used': parameter_names
        }

        return {
            'statusCode': 200,
            'body': json.dumps({
                'success': True,
                'client_name': client_name,
                'credentials': credentials_response,
                'message': f'Successfully retrieved {client_name} credentials from SSM'
            })
        }

    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps({
                'success': False,
                'error': str(e),
                'message': f'Failed to retrieve {client_name} credentials from SSM Parameter Store'
            })
        }

Overwriting lambda_credentials.py


In [26]:
import json
import boto3
from botocore.exceptions import ClientError

def lambda_handler(event, context):
    """Lambda function to retrieve all credentials for a client website
    Args:
        event: Contains client_name for which to retrieve credentials
    Returns:
        Dictionary with all credentials for the requested site
    """
    client_name = event.get('client_name')
    if not client_name:
        return {
            'statusCode': 400,
            'body': json.dumps({
                'success': False,
                'error': 'Missing required parameter: client_name',
                'message': 'Please provide a client_name in the event'
            })
        }

    ssm = boto3.client('ssm')

    def get_secret(parameter_name):
        try:
            response = ssm.get_parameter(Name=parameter_name, WithDecryption=True)
            return response['Parameter']['Value']
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == 'ParameterNotFound':
                raise Exception(f"Parameter '{parameter_name}' not found")
            elif error_code == 'AccessDenied':
                raise Exception(f"Access denied to parameter '{parameter_name}' - check IAM permissions")
            else:
                raise Exception(f"Error retrieving '{parameter_name}': {str(e)}")
        except Exception as e:
            raise Exception(f"Unexpected error retrieving '{parameter_name}': {str(e)}")

    # Construct parameter names dynamically
    login_param = f"{client_name}_Login"
    password_param = f"{client_name}_Password"
    weburl_param = f"{client_name}_WebURL"

    parameter_names = [login_param, password_param, weburl_param]

    try:
        login_credentials = get_secret(login_param)
        password_credentials = get_secret(password_param)
        web_url = get_secret(weburl_param)

        if not all([login_credentials, password_credentials, web_url]):
            missing = []
            if not login_credentials: missing.append(login_param)
            if not password_credentials: missing.append(password_param)
            if not web_url: missing.append(weburl_param)
            raise Exception(f"Missing required parameters: {', '.join(missing)}")

        credentials_response = {
            'starting_url': web_url,
            'login_credentials': login_credentials,
            'login_password': password_credentials,
            'file_info': {
                'upload_path': '/uploads',
                'allowed_extensions': ['.pdf', '.docx', '.txt'],
                'max_file_size': '10MB'
            },
            'retrieved_from': 'AWS_SSM_Parameter_Store',
            'parameters_used': parameter_names
        }

        return {
            'statusCode': 200,
            'body': json.dumps({
                'success': True,
                'client_name': client_name,
                'credentials': credentials_response,
                'message': f'Successfully retrieved {client_name} credentials from SSM'
            })
        }

    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps({
                'success': False,
                'error': str(e),
                'message': f'Failed to retrieve {formatted_client_name} credentials from SSM Parameter Store'
            })
        }


In [ ]:
import boto3
import json

lambda_client = boto3.client('lambda', region_name='us-east-1')
response = lambda_client.invoke(
    FunctionName='your_lambda_function_name',
    InvocationType='RequestResponse',
    Payload=json.dumps({'client_name': 'TechResidential'})
)
result = json.loads(response['Payload'].read())
print(result)

In [40]:
!aws lambda create-function \
  --function-name agentcore-credential-retrieval \
  --runtime python3.9 \
  --role arn:aws:iam::254599367545:role/gateway_lambda_iamrole \
  --handler lambda_credentials.lambda_handler \
  --zip-file fileb://lambda_credentials.zip

{
    "FunctionName": "agentcore-credential-retrieval",
    "FunctionArn": "arn:aws:lambda:us-east-1:254599367545:function:agentcore-credential-retrieval",
    "Runtime": "python3.9",
    "Role": "arn:aws:iam::254599367545:role/gateway_lambda_iamrole",
    "Handler": "lambda_credentials.lambda_handler",
    "CodeSize": 1228,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2025-09-24T22:49:15.707+0000",
    "CodeSha256": "PCcRLre0lbe0TCmQ3Mxg1xQvWjLyfeLqSns6rZmw2G0=",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "0d9cacb1-690d-418b-80bb-24ebb3caea41",
    "State": "Pending",
    "StateReason": "The function is being created.",
    "StateReasonCode": "Creating",
    "PackageType": "Zip",
    "Architectures": [
        "x86_64"
    ],
    "EphemeralStorage": {
        "Size": 512
    },
    "SnapStart": {
        "ApplyOn": "None",
        "OptimizationStatus": "Off"
    },
    "RuntimeVersionC

In [28]:
lambda_handler({'client_name': 'TechResidential'}, "context")

{'statusCode': 200,
 'body': '{"success": true, "client_name": "TechResidential", "credentials": {"starting_url": "https://outlook.office.com/mail", "login_credentials": "hsinwei@techresidential.com", "login_password": "David9709530!!12", "file_info": {"upload_path": "/uploads", "allowed_extensions": [".pdf", ".docx", ".txt"], "max_file_size": "10MB"}, "retrieved_from": "AWS_SSM_Parameter_Store", "parameters_used": ["TechResidential_Login", "TechResidential_Password", "TechResidential_WebURL"]}, "message": "Successfully retrieved TechResidential credentials from SSM"}'}

In [48]:
import json
import boto3
from botocore.exceptions import ClientError

def lambda_handler2(event, context):
    """
    Lambda function to retrieve a prompt file from S3 based on client_name.
    Args:
        event: Contains client_name for which to retrieve the file
    Returns:
        Dictionary with file content or error message
    """
    client_name = event.get('client_name')
    if not client_name:
        return {
            'statusCode': 400,
            'body': json.dumps({
                'success': False,
                'error': 'Missing required parameter: client_name',
                'message': 'Please provide a client_name in the event'
            })
        }

    bucket_name = 'bedrock-web-automation-dev-storage'
    file_key = f'prompt-files/{client_name}/prompt.txt'

    s3 = boto3.client('s3')

    try:
        response = s3.get_object(Bucket=bucket_name, Key=file_key)
        file_content = response['Body'].read().decode('utf-8')

        return {
            'statusCode': 200,
            'body': json.dumps({
                'success': True,
                'client_name': client_name,
                'file_key': file_key,
                'file_content': file_content,
                'message': f'Successfully retrieved file for {client_name} from S3'
            })
        }
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'NoSuchKey':
            error_msg = f"File '{file_key}' not found in bucket '{bucket_name}'"
        elif error_code == 'AccessDenied':
            error_msg = f"Access denied to file '{file_key}' - check IAM permissions"
        else:
            error_msg = f"Error retrieving file '{file_key}': {str(e)}"
        return {
            'statusCode': 500,
            'body': json.dumps({
                'success': False,
                'error': error_msg,
                'message': f'Failed to retrieve file for {client_name} from S3'
            })
        }
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps({
                'success': False,
                'error': str(e),
                'message': f'Unexpected error retrieving file for {client_name} from S3'
            })
        }

In [49]:
lambda_handler2({'client_name': 'TechResidential'}, "context")

{'statusCode': 200,
 'body': '{"success": true, "client_name": "TechResidential", "file_key": "prompt-files/TechResidential/prompt.txt", "file_content": "Sign in to Microsoft OneDrive using the following credentials:\\n\\nEmail: hsinwei@techresidential.com\\nPassword: David9709530\\n\\nAfter signing in, confirm that you have successfully accessed the OneDrive dashboard.\\n", "message": "Successfully retrieved file for TechResidential from S3"}'}

In [52]:
!aws lambda create-function \
  --function-name agentcore-prompt-file-retrieval \
  --runtime python3.9 \
  --role arn:aws:iam::254599367545:role/gateway_lambda_iamrole \
  --handler lambda_get_prompt_file.lambda_handler \
  --zip-file fileb://lambda_get_prompt_file.zip

{
    "FunctionName": "agentcore-prompt-file-retrieval",
    "FunctionArn": "arn:aws:lambda:us-east-1:254599367545:function:agentcore-prompt-file-retrieval",
    "Runtime": "python3.9",
    "Role": "arn:aws:iam::254599367545:role/gateway_lambda_iamrole",
    "Handler": "lambda_get_prompt_file.lambda_handler",
    "CodeSize": 947,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2025-09-25T00:13:37.469+0000",
    "CodeSha256": "cFbRIPlWWUzS6OLbJDHel9VcNEPJ4augOkdEz32+/Ys=",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "502890b4-7e28-4fb0-952c-9b4b5420aa81",
    "State": "Pending",
    "StateReason": "The function is being created.",
    "StateReasonCode": "Creating",
    "PackageType": "Zip",
    "Architectures": [
        "x86_64"
    ],
    "EphemeralStorage": {
        "Size": 512
    },
    "SnapStart": {
        "ApplyOn": "None",
        "OptimizationStatus": "Off"
    },
    "RuntimeVer

In [ ]:
# Set up AWS environment
import os
import boto3

# Set AWS region
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')
REGION = os.environ['AWS_DEFAULT_REGION']

print(f"Using AWS region: {REGION}")

In [ ]:
# Configure AgentCore Gateway to expose Lambda function as a tool
bedrock_agent_client = boto3.client('bedrock-agent', region_name=REGION)

GATEWAY_TARGET_NAME = 'credential-retrieval-tool'

try:
    # Create AgentCore Gateway target configuration
    gateway_target_config = {
        'targetName': GATEWAY_TARGET_NAME,
        'targetType': 'LAMBDA',
        'lambdaConfiguration': {
            'functionArn': function_arn,
            'payloadFormat': 'JSON'
        },
        'description': 'Lambda function to retrieve website credentials for web automation',
        'toolSpec': {
            'name': 'get_credentials',
            'description': 'Retrieve login credentials and file information for website automation',
            'inputSchema': {
                'type': 'object',
                'properties': {
                    'site_name': {
                        'type': 'string',
                        'description': 'Name of the website/service to get credentials for (default, banking, ecommerce)',
                        'enum': ['default', 'banking', 'ecommerce']
                    },
                    'credential_type': {
                        'type': 'string',
                        'description': 'Type of credentials to retrieve',
                        'enum': ['all', 'login_only', 'file_info_only'],
                        'default': 'all'
                    }
                },
                'required': ['site_name']
            }
        }
    }
    
    # Create the Gateway target
    try:
        target_response = bedrock_agent_client.create_gateway_target(**gateway_target_config)
        print(f"Created AgentCore Gateway target: {GATEWAY_TARGET_NAME}")
        target_id = target_response['targetId']
        
    except bedrock_agent_client.exceptions.ConflictException:
        # Target already exists, update it
        # First get the existing target ID
        targets = bedrock_agent_client.list_gateway_targets()
        target_id = None
        for target in targets.get('targets', []):
            if target['targetName'] == GATEWAY_TARGET_NAME:
                target_id = target['targetId']
                break
        
        if target_id:
            gateway_target_config['targetId'] = target_id
            target_response = bedrock_agent_client.update_gateway_target(**gateway_target_config)
            print(f"Updated existing AgentCore Gateway target: {GATEWAY_TARGET_NAME}")
        else:
            raise Exception("Could not find existing target to update")
    
    print(f"Gateway target ID: {target_id}")
    print("✅ AgentCore Gateway target configured successfully")
    
    # Display tool usage example
    print("\n" + "="*60)
    print("AGENTCORE GATEWAY TOOL CONFIGURATION COMPLETE")
    print("="*60)
    print(f"Tool Name: get_credentials")
    print(f"Gateway Target: {GATEWAY_TARGET_NAME}")
    print(f"Lambda Function: {FUNCTION_NAME}")
    print("\nTool Usage Examples:")
    print("1. Get all credentials for default site:")
    print('   get_credentials(site_name="default")')
    print("\n2. Get only login credentials for banking:")
    print('   get_credentials(site_name="banking", credential_type="login_only")')
    print("\n3. Get file info for ecommerce:")
    print('   get_credentials(site_name="ecommerce", credential_type="file_info_only")')
    
except Exception as e:
    print(f"Error configuring AgentCore Gateway: {str(e)}")
    print("Note: Make sure you have AgentCore Gateway permissions and the service is available in your region")

In [ ]:
# Example usage and testing of the complete AgentCore Gateway integration

print("🚀 AGENTCORE GATEWAY INTEGRATION DEMO")
print("="*60)

# Example 1: Test credential retrieval
print("\n1. Testing credential retrieval for different sites...")

test_sites = ['default', 'banking', 'ecommerce']
for site in test_sites:
    print(f"\n🔍 Testing credentials for: {site}")
    try:
        # This would normally be called by the agent through AgentCore Gateway
        # For demo purposes, we'll simulate the call
        test_payload = {'site_name': site, 'credential_type': 'all'}
        response = lambda_client.invoke(
            FunctionName=FUNCTION_NAME,
            InvocationType='RequestResponse',
            Payload=json.dumps(test_payload)
        )
        
        result = json.loads(response['Payload'].read())
        if result.get('statusCode') == 200:
            body = json.loads(result['body'])
            creds = body['credentials']
            print(f"   ✅ URL: {creds.get('starting_url')}")
            print(f"   ✅ Login: {creds.get('login_credentials')}")
            print(f"   ✅ File upload path: {creds.get('file_info', {}).get('upload_path')}")
        else:
            print(f"   ❌ Failed to retrieve credentials")
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")

# Example 2: Agent task automation scenarios
print(f"\n2. Agent Task Automation Scenarios:")
print("-" * 40)

automation_scenarios = [
    {
        'task': 'Upload financial document to banking portal',
        'site': 'banking',
        'expected_actions': ['login', 'navigate to upload', 'select file', 'upload']
    },
    {
        'task': 'Extract product information from ecommerce site',
        'site': 'ecommerce', 
        'expected_actions': ['login', 'navigate to products', 'extract data']
    },
    {
        'task': 'Navigate to account settings and update profile',
        'site': 'default',
        'expected_actions': ['login', 'navigate to settings', 'update profile']
    }
]

for i, scenario in enumerate(automation_scenarios, 1):
    print(f"\nScenario {i}: {scenario['task']}")
    print(f"   Target site: {scenario['site']}")
    print(f"   Expected actions: {' → '.join(scenario['expected_actions'])}")
    print(f"   Integration flow:")
    print(f"     1. Agent calls retrieve_credentials(site_name='{scenario['site']}')")
    print(f"     2. AgentCore Gateway routes to Lambda function")
    print(f"     3. Lambda returns secure credentials")
    print(f"     4. Agent uses Nova Act + AgentCore Browser Tool")
    print(f"     5. Automated task execution with retrieved credentials")

# Example 3: Show the complete integration architecture
print(f"\n3. Complete Integration Architecture:")
print("=" * 50)
print("""
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│   Strands       │    │   AgentCore      │    │   Lambda        │
│   Agent         │───▶│   Gateway        │───▶│   Function      │
│                 │    │                  │    │                 │
│ @tool           │    │ Target:          │    │ Credential      │
│ retrieve_creds()│    │ get_credentials  │    │ Store           │
└─────────────────┘    └──────────────────┘    └─────────────────┘
        │                                               │
        ▼                                               ▼
┌─────────────────┐                              ┌─────────────────┐
│   Nova Act      │                              │   Secure        │
│   Browser       │                              │   Response      │
│   Automation    │◀─────────────────────────────│   • URLs        │
│                 │         Credentials          │   • Login Info  │
│   • Navigate    │                              │   • File Paths  │
│   • Login       │                              │   • Metadata    │
│   • Interact    │                              └─────────────────┘
└─────────────────┘
""")

print("\n4. Key Benefits of This Architecture:")
print("-" * 40)
print("✅ Secure credential separation from automation logic")
print("✅ AgentCore Gateway provides unified tool interface") 
print("✅ Lambda function enables scalable credential management")
print("✅ Supports multiple sites with different credential types")
print("✅ Easy to extend with additional authentication methods")
print("✅ Nova Act integration for robust browser automation")
print("✅ Monitoring and logging through AgentCore Observability")

print(f"\n5. Deployment Summary:")
print("=" * 30)
print(f"✅ Lambda Function: {FUNCTION_NAME}")
print(f"✅ IAM Role: {ROLE_NAME}")
print(f"✅ Gateway Target: {GATEWAY_TARGET_NAME}")
print(f"✅ Region: {REGION}")
print("✅ Agent Tools: retrieve_credentials, automate_web_task")

print(f"\n6. Next Steps for Production:")
print("-" * 35)
print("1. Replace mock credentials with AWS Secrets Manager")
print("2. Add proper error handling and retry logic")
print("3. Implement credential encryption at rest")
print("4. Set up CloudWatch monitoring and alerts")
print("5. Add authentication for Lambda function access")
print("6. Configure VPC and security groups for network isolation")
print("7. Implement audit logging for credential access")
print("8. Add support for dynamic credential rotation")

print("\n🎉 AgentCore Gateway integration is complete and ready for use!")
print("The agent can now securely retrieve credentials and perform web automation.")